In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from matplotlib.font_manager import FontProperties

In [ ]:
df = pd.read_csv("result.csv")

In [ ]:
baseline = df[df.with_fla == False].groupby(['source_model', 'target_model','fool_method'])['acc_after_attack'].mean().reset_index()
baseline

In [ ]:
q = df[(df.with_fla == True)].groupby(['source_model', 'target_model','fool_method','layer_index'])['acc_after_attack'].mean().reset_index()
q
# q[(q.source_model == 'ResNet18') & (q.target_model != 'ResNet18')]


In [ ]:
layer_selection = {'DenseNet121': 7, 'GoogLeNet': 9,'ResNet18': 4,'SENet18': 4}
# select = q[q.layer_index == layer_selection[q.source_model]]
# select
layer_selected = q[q.layer_index == q.source_model.apply(lambda a : layer_selection[a])].reset_index()
layer_selected = layer_selected.drop(['index'], axis=1)
layer_selected

In [ ]:
optimum = q.groupby(['source_model','target_model','fool_method']).apply(lambda x: x[x.acc_after_attack == x.acc_after_attack.min()][['layer_index', 'acc_after_attack']]).reset_index()
optimum = optimum.drop(['level_3'], axis=1)
optimum


In [ ]:
compare = layer_selected.merge(optimum, on = ['source_model','target_model','fool_method'], suffixes = ["","_opt"])
compare = compare.merge(baseline, on = ['source_model','target_model','fool_method'], suffixes = ["","_baseline"])
compare

In [ ]:
def visualize_result(name):
    df = pd.read_csv(name)
    source_models = df.source_model.unique()
    target_models =  df.target_model.unique()
    attacks = df.fool_method.unique()
    metrics = ['acc_after_attack']
    target_models = ['ResNet18', 'DenseNet121', 'GoogLeNet', 'SENet18']
    for metric in metrics:
        
        for source_model in source_models:
#             fig = plt.figure()
            fig = plt.figure(figsize = [6*len(attacks), 4])
            
            for i, attack in enumerate(attacks):
                
                plt.subplot(1,len(attacks),i+1)
                
                layer_indexs = df[df.source_model == source_model]['layer_index'].unique()[1:]
                layer_names = df[df.source_model == source_model]['layer_name'].unique()[1:]
                print(layer_names)
                for target_model in target_models:           
                    r = df[(df.source_model == source_model) & (df.target_model == target_model) & (df.fool_method == attack)]
                    
                    fla = r[r.with_fla].groupby('layer_index')
                    other = r[r.with_fla == False]
#                     print(other)
                    xs = fla.layer_index.unique()
                    baseline = other[metric].mean()
                    fla_r = fla[metric].mean()
                    names = fla.layer_name.unique()
#                     print(source_model)
#                     print(target_model)
#                     print(attack)
#                     print([baseline for i in xs])
                    p = plt.plot(layer_indexs,  [baseline for i in xs], linestyle = '--', label = target_model)
                    plt.plot(fla_r, label =  target_model + "(ILA proj_loss)", color = p[0]._color)
#                     plt.xticks(layer_indexs, layer_names)
                    if source_model != target_model and ((metric == 'fool_rate' and not(fla_r > baseline).any())or (metric == 'acc_after_attack' and not(fla_r < baseline).any())):
                        print("never perform absolutely better: " + source_model + " " + target_model)
                
                
                plt.ylabel('Accuracy after Attack')
                plt.xlabel('Layer Index')
                fontP = FontProperties()
                fontP.set_size('small')
                
                plt.legend(ncol = 3, loc=[0.05, -0.4], prop=fontP)
#                 if attack == 'ifgsm':
#                     plt.title( "ifgsm, source:" + source_model)
#                 if attack == 'momentum_ifgsm':
#                     plt.title("momentum_ifgsm, source:" + source_model )
             
#                 plt.title(attack + " source:" + source_model)
#                 plt.title(name[-16:-4])
            plt.show()
            fig.savefig( name[:-4] + '.pdf')
    
#             fig.savefig( 'project_loss_sign_50batches_{}_source:{}.pdf'.format(attack, source_model))


In [ ]:
visualize_result('result.csv')